# 🏦 Sequential Workflow for Loan Application Processing

## Overview

This notebook demonstrates **sequential orchestration** using the `SequentialBuilder` API for a **loan application processing** workflow. Each agent processes the application in sequence, passing context to the next agent.

### 💼 Industry Use Case: Loan Application Pipeline

A customer submits a loan application. The workflow processes it through two specialized agents:
1. **Loan Analyst**: Reviews the application, checks eligibility criteria
2. **Risk Reviewer**: Evaluates risk factors and provides final recommendation

### ⚠️ Important Financial Disclaimer
> **This notebook is for educational and demonstration purposes only.** The loan processing logic shown here is simplified and should not be used for actual lending decisions. Always consult with licensed financial professionals and follow regulatory requirements.

### Key Concepts

| Concept | Description |
|---------|-------------|
| **SequentialBuilder** | High-level API for building linear agent workflows |
| **Shared Context** | `list[ChatMessage]` flows through all participants |
| **Incremental Building** | Each agent sees previous messages and builds on them |
| **Final Output** | Complete conversation history with all agent responses |

### Architecture

```
Loan Application
    ↓
[input-conversation adapter]
    ↓
Loan Analyst Agent (eligibility check)
    ↓
[to-conversation:analyst adapter]
    ↓
Risk Reviewer Agent (risk assessment)
    ↓
[to-conversation:reviewer adapter]
    ↓
[complete adapter]
    ↓
Final Loan Decision (list[ChatMessage])
```

## Prerequisites

- ✅ Azure OpenAI Service configured
- ✅ Environment variables: `AZURE_OPENAI_ENDPOINT`, `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- ✅ Azure CLI authentication: Run `az login` before executing

## 1️⃣ Setup and Imports

Import required libraries for building the sequential loan processing workflow.

In [ ]:
import asyncio
from typing import cast

import os
from dotenv import load_dotenv
from azure.identity import AzureCliCredential

from agent_framework import ChatMessage, Role, SequentialBuilder, WorkflowOutputEvent
from agent_framework.azure import AzureOpenAIChatClient

# Load environment variables from .env file
load_dotenv('../../.env')

# Verify environment is loaded
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
print(f"✅ Environment loaded: {azure_endpoint is not None and deployment_name is not None}")

## 2️⃣ Create Loan Processing Agents

### 🏦 Loan Analyst Agent
- Reviews loan application details
- Checks basic eligibility criteria (income, employment, credit)
- First in the sequence - creates initial assessment

### 📊 Risk Reviewer Agent
- Evaluates risk factors identified by the analyst
- Provides final recommendation (approve/decline/conditional)
- Second in the sequence - sees both application and analyst's assessment

In [ ]:
# Create Azure OpenAI chat client
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
chat_client = AzureOpenAIChatClient(
    deployment_name=deployment_name,
    endpoint=endpoint,
    credential=AzureCliCredential()
)
print("✅ Azure OpenAI Chat Client created")

# Loan Analyst Agent - Reviews applications and checks eligibility
loan_analyst = chat_client.as_agent(
    instructions=(
        "You are a Loan Analyst at a retail bank. Your role is to review loan applications and assess basic eligibility.\n"
        "When reviewing an application:\n"
        "1. Check if income meets minimum requirements (typically 3x monthly payment)\n"
        "2. Verify employment stability\n"
        "3. Note any red flags or concerns\n"
        "4. Provide a preliminary assessment\n"
        "Always include a disclaimer that this is a preliminary review and final decisions require additional verification."
    ),
    name="loan_analyst",
)
print("✅ Loan Analyst Agent created")

# Risk Reviewer Agent - Evaluates risk and makes recommendations
risk_reviewer = chat_client.as_agent(
    instructions=(
        "You are a Risk Reviewer at a retail bank. You evaluate loan applications based on the analyst's assessment.\n"
        "Your role:\n"
        "1. Review the analyst's findings\n"
        "2. Assess overall risk level (Low/Medium/High)\n"
        "3. Provide a recommendation: APPROVE, DECLINE, or CONDITIONAL APPROVAL\n"
        "4. List any conditions or requirements if conditional\n"
        "Keep your response concise and structured. Include standard compliance disclaimers."
    ),
    name="risk_reviewer",
)
print("✅ Risk Reviewer Agent created")

## 3️⃣ Build Sequential Workflow

The `SequentialBuilder` creates a linear pipeline where each agent processes the conversation in order.

**Execution Flow:**
1. User submits loan application → Loan Analyst reviews
2. Analyst's assessment added to conversation → Risk Reviewer evaluates
3. Complete conversation with both assessments returned

In [ ]:
# Build sequential workflow: loan_analyst -> risk_reviewer
workflow = SequentialBuilder().participants([loan_analyst, risk_reviewer]).build()
print("✅ Sequential workflow built: Loan Analyst → Risk Reviewer")

## 4️⃣ Process Loan Application

Submit a sample loan application and watch both agents process it in sequence.

### Sample Application Details
- **Applicant**: John Smith
- **Loan Amount**: $25,000 auto loan
- **Annual Income**: $65,000
- **Employment**: 3 years at current employer
- **Purpose**: Purchase used vehicle

In [ ]:
# Sample loan application
loan_application = """
LOAN APPLICATION
================
Applicant: John Smith
Loan Type: Auto Loan
Requested Amount: $25,000
Loan Term: 48 months

FINANCIAL INFORMATION:
- Annual Income: $65,000
- Monthly Expenses: $2,500
- Existing Debt: $8,000 (credit cards)
- Credit Score: 720

EMPLOYMENT:
- Employer: TechCorp Inc.
- Position: Software Developer
- Employment Duration: 3 years

PURPOSE: Purchase 2022 Honda Accord
"""

print("📋 Processing Loan Application...")
print("=" * 60)

# Run and collect outputs using streaming
outputs: list[list[ChatMessage]] = []
async for event in workflow.run_stream(loan_application):
    if isinstance(event, WorkflowOutputEvent):
        outputs.append(cast(list[ChatMessage], event.data))

if outputs:
    print("\n" + "=" * 60)
    print("📄 LOAN PROCESSING RESULTS")
    print("=" * 60)
    for i, msg in enumerate(outputs[-1], start=1):
        name = msg.author_name or ("assistant" if msg.role == Role.ASSISTANT else "user")
        role_emoji = "👤" if msg.role == Role.USER else "🤖"
        print(f"\n{'-' * 60}")
        print(f"{role_emoji} [{name.upper()}]")
        print(f"{'-' * 60}")
        print(f"{msg.text}")

print("\n" + "=" * 60)
print("✅ Loan application processing complete!")

## 📝 Key Takeaways

### Sequential Workflow Benefits for FSI

| Benefit | Description |
|---------|-------------|
| **Audit Trail** | Complete conversation history shows decision process |
| **Separation of Duties** | Different agents handle different aspects (analyst vs reviewer) |
| **Compliance** | Each step documented in the conversation |
| **Scalability** | Easy to add more agents (compliance check, manager approval) |

### When to Use Sequential Workflows in Banking

- **Loan Processing**: Application → Analysis → Risk Review → Decision
- **Account Opening**: KYC → Compliance → Approval
- **Transaction Review**: Detection → Analysis → Escalation
- **Credit Assessment**: Data Collection → Scoring → Recommendation

